In [38]:
from sklearn.linear_model import LogisticRegression
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.learning_curve import validation_curve
import datetime

In [2]:
train_path = os.path.join('..','data','train_data_v3_processed.csv')
train_data = pd.read_csv(train_path)

In [30]:
test_path = os.path.join('..','data','test_data_v3_processed.csv')
test_data = pd.read_csv(test_path)

In [3]:
df = train_data#[:10000]

In [44]:
df.columns

Index(['id', 'question1', 'question2', 'is_duplicate', 'question1_lemma',
       'question1_tag', 'question2_lemma', 'question2_tag', 'fuzz_ratio',
       'fuzz_partial_ratio', 'fuzz_partial_token_set_ratio',
       'fuzz_partial_token_sort_ratio', 'fuzz_token_set_ratio',
       'fuzz_token_sort_ratio', 'len_q1', 'len_q2', 'diff_len', 'len_char_q1',
       'len_char_q2', 'len_word_q1', 'len_word_q2', 'common_words',
       'avg_word_q1', 'avg_word_q2', 'num_stop_words_q1', 'num_stop_words_q2',
       'numerics_q1', 'numerics_q2', 'uppercase_q1', 'uppercase_q2',
       'softcossim', 'mistakes_q1', 'mistakes_q2', 'spacy_sim',
       'q1_q2_intersect', 'q1_freq', 'q2_freq', 'word_match', 'tfidf_wm',
       'tfidf_wm_stops', 'jaccard', 'wc_diff', 'wc_ratio', 'wc_diff_unique',
       'wc_ratio_unique', 'wc_diff_unq_stop', 'wc_ratio_unique_stop',
       'same_start', 'char_diff', 'char_diff_unq_stop', 'total_unique_words',
       'total_unq_words_stop', 'char_ratio'],
      dtype='object')

In [4]:
df.head()

,id,question1,question2,is_duplicate,question1_lemma,question1_tag,question2_lemma,question2_tag,fuzz_ratio,fuzz_partial_ratio,...,wc_diff_unique,wc_ratio_unique,wc_diff_unq_stop,wc_ratio_unique_stop,same_start,char_diff,char_diff_unq_stop,total_unique_words,total_unq_words_stop,char_ratio
0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,what be the step by step guide to invest in sh...,WP VBZ DT NN IN NN NN TO VB IN NN NN IN NN .,what be the step by step guide to invest in sh...,WP VBZ DT NN IN NN NN TO VB IN NN NN .,93,98,...,1,0.916667,1,0.833333,1,7,5,13,7,0.867925
1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,what be the story of kohinoor ( koh - i - noor...,WP VBZ DT NN IN NNP -LRB- NNP HYPH NNP HYPH NN...,what would happen if the indian government ste...,WP MD VB IN DT JJ NN VBD DT NNP -LRB- NNP HYPH...,68,78,...,4,1.500000,5,2.250000,1,32,31,16,11,1.727273
2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,how can -PRON- increase the speed of -PRON- in...,WRB MD PRP VB DT NN IN PRP$ NN NN IN VBG DT NNP .,how can internet speed be increase by hack thr...,WRB MD VB NN VB VBN IN VBG IN NN .,35,44,...,4,0.714286,1,0.833333,1,10,7,20,9,0.833333
3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,why be -PRON- mentally very lonely ? how can -...,WRB VBP PRP RB RB JJ . WRB MD PRP VB PRP .,find the remainder when [ math]23^{24}[/math ]...,VB DT NN WRB -LRB- NN -RRB- VBZ VBN IN CD .,17,22,...,1,0.900000,1,1.250000,0,17,23,19,9,1.425000
4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"which one dissolve in water quikly sugar , sal...","WDT CD NN IN NN RB NN , NN , NN CC NN FW NN .",which fish would survive in salt water ?,WDT NN MD VB IN NN NN .,37,55,...,6,0.538462,5,0.500000,1,31,28,18,15,0.515625


In [5]:
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60.
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [32]:
x_columns = ['fuzz_ratio',
       'fuzz_partial_ratio', 'fuzz_partial_token_set_ratio',
       'fuzz_partial_token_sort_ratio', 'fuzz_token_set_ratio',
       'fuzz_token_sort_ratio', 'len_q1', 'len_q2', 'diff_len', 'len_char_q1',
       'len_char_q2', 'len_word_q1', 'len_word_q2', 'common_words',
       'avg_word_q1', 'avg_word_q2', 'num_stop_words_q1', 'num_stop_words_q2',
       'numerics_q1', 'numerics_q2', 'uppercase_q1', 'uppercase_q2',
       'softcossim', 'mistakes_q1', 'mistakes_q2', 'spacy_sim',
       'q1_q2_intersect', 'q1_freq', 'q2_freq', 'word_match', 'tfidf_wm',
       'tfidf_wm_stops', 'jaccard', 'wc_diff', 'wc_ratio', 'wc_diff_unique',
       'wc_ratio_unique', 'wc_diff_unq_stop', 'wc_ratio_unique_stop',
       'same_start', 'char_diff', 'char_diff_unq_stop', 'total_unique_words',
       'total_unq_words_stop', 'char_ratio']

y_columns = ['is_duplicate']
df.fillna(0,inplace=True)
test_data.fillna(0,inplace=True)

data_y = df[y_columns].values
data_x = df[x_columns].values
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.20, random_state=42)
#train = train.T
#test = test.T

In [7]:
train_global_df = pd.DataFrame(np.concatenate((train_x, train_y), axis=1))
train_global_df.columns = x_columns + y_columns

data_y = train_global_df[y_columns].values
data_x = train_global_df[x_columns].values

train_x, validation_x, train_y, validation_y = train_test_split(data_x, data_y, test_size=0.20, random_state=42)

train_y = train_y.astype('int')
validation_y = validation_y.astype('int')

In [8]:
C_param_range = [0.001,0.01,0.1,1,10,100,1000]

accuracy_table = pd.DataFrame(columns = ['C_parameter','Training Accuracy', 'Testing Accuracy'])
accuracy_table['C_parameter'] = C_param_range

j = 0
for i in C_param_range:
    start_time = datetime.datetime.now()
    
    lr = LogisticRegression(penalty = 'l2', C = i,random_state = 42)
    lr.fit(train_x,train_y)
    validation_y_pred = lr.predict(validation_x)
    
    accuracy_table.iloc[j,1] = accuracy_score(validation_y,validation_y_pred)
    
    
    test_y_pred = lr.predict(test_x)
    accuracy_table.iloc[j,2] = accuracy_score(test_y,test_y_pred)
    j += 1
    
    end_time = datetime.datetime.now()
    seconds_elapsed = (end_time - start_time).total_seconds()
    print("C = ",i,"It took {} to execute this".format(hms_string(seconds_elapsed)))
    
accuracy_table['Diff'] = np.abs(accuracy_table.iloc[:, 1]-accuracy_table.iloc[:, 2])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


C =  0.001 It took 0:00:14.35 to execute this
C =  0.01 It took 0:00:18.62 to execute this
C =  0.1 It took 0:00:41.06 to execute this
C =  1 It took 0:01:02.19 to execute this
C =  10 It took 0:01:06.56 to execute this
C =  100 It took 0:01:00.58 to execute this
C =  1000 It took 0:01:21.16 to execute this


In [10]:
accuracy_table.head()

,C_parameter,Training Accuracy,Testing Accuracy,Diff
0,0.001,0.856377,0.853619,0.00275802
1,0.010,0.859358,0.85749,0.00186759
2,0.100,0.85959,0.857784,0.00180565
3,1.000,0.859861,0.857552,0.00230891
4,10.000,0.859861,0.857273,0.00258765


In [26]:
bestC = accuracy_table[(accuracy_table['Diff'] == accuracy_table['Diff'].min())]['C_parameter'].values.ravel()[0]

In [27]:
bestC

0.1

In [ ]:
lr = LogisticRegression(penalty = 'l2', C = bestC, random_state = 42)
lr.fit(data_x,data_y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [33]:
predicted = lr.predict(test_data[x_columns].values)

In [42]:

import csv
import datetime
from os.path import join,abspath,curdir
import numpy as np
import pandas as pd

def create_csvs(predicted, test_ids):
    EXPECTED_ROWS = 81126 
    tests_ids_len = len(test_ids)
    assert(tests_ids_len == EXPECTED_ROWS)
    assert(len(predicted)==tests_ids_len)
    
    FOLDER = join('..','data', 'submissions')
    CURRENT_PATH = abspath(curdir)
    
    merged = {'test_id': test_ids}
    merged['is_duplicate'] = predicted

    FILENAME = 'submission_' + datetime.datetime.now().strftime("%I%M%p-%B-%d-%Y") + '.csv'
    df = pd.DataFrame.from_dict(merged)

    df.set_index('test_id', inplace=True)

    FULL_PATH = join(CURRENT_PATH, FOLDER, FILENAME)

    df.to_csv(path_or_buf=FULL_PATH, sep=',')

    print('saved in: ', FULL_PATH)

In [49]:
create_csvs(predicted.astype('int'), test_data.test_id.values)

saved in:  E:\Dropbox\folch\UVA\Applied Machine Learning\Project_quora\models\..\data\submissions\submission_0115PM-December-11-2018.csv
